## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-19-16-58-49 +0000,nyt,Prince Harry’s Court Case Against Daily Mail P...,https://www.nytimes.com/2026/01/19/world/europ...
1,2026-01-19-16-56-00 +0000,nypost,"Woman kills boy, 6, in Pacific Beach hit-and-r...",https://nypost.com/2026/01/19/us-news/6-year-o...
2,2026-01-19-16-51-00 +0000,wsj,Trump Links Greenland Threats to Missing Out o...,https://www.wsj.com/world/europe/trump-links-g...
3,2026-01-19-16-50-36 +0000,nyt,Elon Musk Spends $10 Million to Help Nate Morr...,https://www.nytimes.com/2026/01/19/us/politics...
4,2026-01-19-16-46-06 +0000,nypost,Kristi Noem announces more than 10K arrests of...,https://nypost.com/2026/01/19/us-news/kristi-n...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2416/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,60
19,greenland,27
58,high,13
72,new,13
15,crash,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
87,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...,126
264,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...,125
236,2026-01-18-21-05-32 +0000,bbc,EU weighs response to Trump's tariff threat ov...,https://www.bbc.com/news/articles/c87r41j4n4jo...,118
188,2026-01-19-02-38-00 +0000,wsj,Precious Metals Climb as Trump Plans Tariff on...,https://www.wsj.com/finance/precious-metals-cl...,117
2,2026-01-19-16-51-00 +0000,wsj,Trump Links Greenland Threats to Missing Out o...,https://www.wsj.com/world/europe/trump-links-g...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
87,126,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...
211,82,2026-01-19-00-00-06 +0000,nypost,At least 20 killed in high-speed train derailm...,https://nypost.com/2026/01/18/world-news/at-le...
76,44,2026-01-19-13-00-00 +0000,wsj,Half of U.S. voters say the Trump administrati...,https://www.wsj.com/politics/policy/voters-ske...
72,41,2026-01-19-13-17-00 +0000,wsj,European Stock Markets Tumble on Trump’s Tarif...,https://www.wsj.com/finance/stocks/global-equi...
262,38,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
40,35,2026-01-19-14-52-09 +0000,nypost,Police probe if dead backpacker was ‘attacked’...,https://nypost.com/2026/01/19/world-news/polic...
241,30,2026-01-18-20-13-23 +0000,nypost,Hundreds of thousands of french fries wash up ...,https://nypost.com/2026/01/18/world-news/hundr...
163,30,2026-01-19-06-40-23 +0000,bbc,China hits growth goal after exports defy US t...,https://www.bbc.com/news/articles/cgk8zd287myo...
225,28,2026-01-18-22-10-53 +0000,nypost,Some of Iran’s richest fled to Turkey to party...,https://nypost.com/2026/01/18/world-news/some-...
18,27,2026-01-19-16-04-32 +0000,startribune,"What to watch this week: Hockey Day Minnesota,...",https://www.startribune.com/mn-high-school-bas...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
